In [2]:
import pandas as pd
import requests
import psycopg2
import os
from dotenv import load_dotenv
from time import sleep

# Carrega variáveis de ambiente
load_dotenv()
db_url = os.getenv("DATABASE_URL")
api_key = os.getenv("OPENCAGE_KEY")  # adicione isso ao seu .env também

def geocodificar(cidade, uf):
    try:
        url = f"https://api.opencagedata.com/geocode/v1/json?q={cidade},+{uf},+Brasil&key={api_key}&language=pt&limit=1"
        response = requests.get(url, timeout=10)
        data = response.json()
        if data['results']:
            coords = data['results'][0]['geometry']
            return coords['lat'], coords['lng']
    except:
        pass
    return None, None

# 1. Lê a planilha


In [3]:
df = pd.read_excel("App_transportadora.xlsx")
df.columns = df.columns.str.lower().str.strip()
df



,cidade_origem,uf_origem,nome_grupo,transportadora,empresa,contato,ja_carregamos,temos_cadastro,produto,preco,latitude,longitude
0,Paranaguá,PR,GUILHERME RODRIGUES- MAION,TRANSMAION,NaN,Formiga Transmaion Paranaguá 4f,Não,Não,NaN,NaN,NaN,NaN
1,Paranaguá,PR,LUIZ GABRIEL FRETES,TRANSMAION,NaN,NaN,Não,Não,NaN,NaN,NaN,NaN
2,Luziania,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,NaN,NaN,NaN
3,Ctristanila,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,NaN,NaN,NaN
4,Goiatuba,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,Soja,241.0,NaN,NaN
5,Bom Jesus,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,51.0,NaN,NaN
6,Marcianopolis,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,51.0,NaN,NaN
7,Santa Helena,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,270.0,NaN,NaN
8,Rio Verd,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,298.0,NaN,NaN
9,Nova Crixas,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,340.0,NaN,NaN


In [5]:
# 2. Preenche latitude e longitude

latitudes, longitudes = [], []
for _, row in df.iterrows():
    lat, lon = geocodificar(row['cidade_origem'], row['uf_origem'])
    latitudes.append(lat)
    longitudes.append(lon)
    sleep(1.2)  # evita ultrapassar limites da API gratuita

df["latitude"] = latitudes
df["longitude"] = longitudes

# Remove linhas sem coordenadas


In [6]:
df

,cidade_origem,uf_origem,nome_grupo,transportadora,empresa,contato,ja_carregamos,temos_cadastro,produto,preco,latitude,longitude
0,Paranaguá,PR,GUILHERME RODRIGUES- MAION,TRANSMAION,NaN,Formiga Transmaion Paranaguá 4f,Não,Não,NaN,NaN,-25.514882,-48.522669
1,Paranaguá,PR,LUIZ GABRIEL FRETES,TRANSMAION,NaN,NaN,Não,Não,NaN,NaN,-25.514882,-48.522669
2,Luziania,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,NaN,-16.253546,-47.950028
3,Ctristanila,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,NaN,-15.580710,-49.636230
4,Goiatuba,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,Soja,241.0,-18.015458,-49.364059
5,Bom Jesus,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,51.0,-15.763353,-49.158465
6,Marcianopolis,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,51.0,-17.966373,-50.003908
7,Santa Helena,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,270.0,-18.408417,-49.276975
8,Rio Verd,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,298.0,-15.580710,-49.636230
9,Nova Crixas,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,340.0,-14.096885,-50.339601


In [8]:

# 3. Conecta ao PostgreSQL
conn = psycopg2.connect(db_url)
cursor = conn.cursor()

# 4. Limpa a tabela
cursor.execute("DELETE FROM app_transportadoras")

In [9]:
df

,cidade_origem,uf_origem,nome_grupo,transportadora,empresa,contato,ja_carregamos,temos_cadastro,produto,preco,latitude,longitude
0,Paranaguá,PR,GUILHERME RODRIGUES- MAION,TRANSMAION,NaN,Formiga Transmaion Paranaguá 4f,Não,Não,NaN,NaN,-25.514882,-48.522669
1,Paranaguá,PR,LUIZ GABRIEL FRETES,TRANSMAION,NaN,NaN,Não,Não,NaN,NaN,-25.514882,-48.522669
2,Luziania,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,NaN,-16.253546,-47.950028
3,Ctristanila,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,NaN,-15.580710,-49.636230
4,Goiatuba,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,Soja,241.0,-18.015458,-49.364059
5,Bom Jesus,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,51.0,-15.763353,-49.158465
6,Marcianopolis,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,51.0,-17.966373,-50.003908
7,Santa Helena,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,270.0,-18.408417,-49.276975
8,Rio Verd,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,298.0,-15.580710,-49.636230
9,Nova Crixas,GO,NaN,1500,NaN,ZAMBAK,Sim,Sim,NaN,340.0,-14.096885,-50.339601


In [10]:


# Inserção no banco
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO app_transportadoras (
            cidade_origem, uf_origem, nome_grupo, transportadora,
            empresa, contato, ja_carregamos, temos_cadastro,
            produto, preco, latitude, longitude
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['cidade_origem'], row['uf_origem'], row['nome_grupo'], row['transportadora'],
        row['empresa'], row['contato'], row['ja_carregamos'], row['temos_cadastro'],
        row['produto'], row['preco'], row['latitude'], row['longitude']
    ))


conn.commit()
cursor.close()
conn.close()

print("✅ Dados atualizados com sucesso no banco PostgreSQL.")


✅ Dados atualizados com sucesso no banco PostgreSQL.
